

特征选择；
决策树生产
决策树修剪

决策树模型易于解释 - if-then

In [1]:
my_data=[['slashdot','USA','yes',18,'None'],
         ['google','France','yes',23,'Premium'],
         ['digg','USA','yes',24,'Basic'],
         ['kiwitobes','France','yes',23,'Basic'],
         ['google','UK','no',21,'Premium'],
         ['(direct)','New Zealand','no',12,'None'],
         ['(direct)','UK','no',21,'Basic'],
         ['google','USA','no',24,'Premium'],
         ['slashdot','France','yes',19,'None'],
         ['digg','USA','no',18,'None'],
         ['google','UK','no',18,'None'],
         ['kiwitobes','UK','no',19,'None'],
         ['digg','New Zealand','yes',12,'Basic'],
         ['slashdot','UK','no',21,'None'],
         ['google','UK','yes',18,'Basic'],
         ['kiwitobes','France','yes',19,'Basic']]

In [ ]:
#my_data  = [line.split('\t') for line in open('decisionn_tree_example.txt')]

In [2]:
# node represent node in tree
class decisionnode:
    def __init__(self, col=-1, value=None, results=None, tb=None, fb=None):
        self.col = col # the criteria to be tested index
        self.value = value
        self.results = results
        self.tb = tb  # true branch
        self.fb = fb  # false branch

In [ ]:
# CART: 
## 1. build root point, then select obsever value, divide the data

In [3]:
def divideset(rows, column, value):
    # define split funciton return true or false
    split_function = None
    if isinstance(value, int) or isinstance(value, float):
        split_function = lambda row: row[column] >= value
    else:
        split_function = lambda row: row[column] == value
        
    # divide to two sets
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    
    return set1, set2

In [4]:
divideset(my_data, 3, 20) # divide by column 4, >20 or not

([['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['slashdot', 'UK', 'no', 21, 'None']],
 [['slashdot', 'USA', 'yes', 18, 'None'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']])

In [5]:
divideset(my_data, 2, 'yes') # column 2 is classify parameater

([['slashdot', 'USA', 'yes', 18, 'None'],
  ['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']],
 [['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None'],
  ['slashdot', 'UK', 'no', 21, 'None']])

In [ ]:
## choosing the best split

In [6]:
# count last column result
def uniquecounts(rows):
    results = {}
    for row in rows:
        r = row[len(row)-1] # last column is result: None/basic..
        if r not in results: results[r] = 0
        results[r] += 1
    return results

In [7]:
results = uniquecounts(my_data)
results

{'Basic': 6, 'None': 7, 'Premium': 3}

In [8]:
# gini dex  = Gini(p) = sum(p_k(1-p_k))
def giniimpurity(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp = 0
    for k1 in counts:
        p1 = float(counts[k1])/total # cal very selectipn probility
        for k2 in counts:
            if k1 == k2: continue # try to adjus it same or not
            p2 = float(counts[k2])/total
            imp += p1 * p2
    return imp

In [9]:
giniimpurity([[1],[2],[3],[4]]) # four choices, so you have 75% select you want select.

0.75

In [10]:
giniimpurity(my_data)

0.6328125

In [11]:
# sum of p(x)*log(p(x))

from math import log

def entropy(rows):
    log2 = lambda x: log(x)/log(2)
    results = uniquecounts(rows)
    ent = 0.0
    for r in results.keys():
        p = float(results[r]) / len(rows)
        ent = ent - p*log2(p)
    return ent

In [12]:
entropy(my_data) 

1.5052408149441479

In [14]:
set1, set2 = divideset(my_data, 2, 'yes')
entropy(set1)

1.2987949406953985

In [15]:
entropy(set2)

1.2987949406953985

In [16]:
giniimpurity(set2)

0.53125

In [17]:
# build tree use recursive

def buildtree(rows, scoref=entropy):
    if len(rows) == 0: return decisionnode()  # node can not be divide
    current_score = scoref(rows)
    
    # define variance as split condition
    best_gain = 0.0  # =0, means can not divide 
    best_criteria = None
    best_sets = None
    
    column_count = len(rows[0]) - 1
    for col in range(0, column_count): # last column is result
        column_values = {}
        for row in rows:
            column_values[row[col]] = 1
        # Now try dividing the rows up for each value
        for value in column_values.keys():
            (set1, set2) = divideset(rows, col, value)
            
            #informatin gain
            p = float(len(set1)) / len(rows)
            # H(p) = -p*logp - (1-p)log(1-p)
            gain = current_score - p * scoref(set1) - (1 - p) * scoref(set2)
            if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1, set2)
                
    if best_gain > 0:
        trueBranch = buildtree(best_sets[0])
        falseBranch = buildtree(best_sets[1])
        return decisionnode(col=best_criteria[0], value=best_criteria[1], 
                            tb=trueBranch, fb=falseBranch)
    else:
        return decisionnode(results=uniquecounts(rows))

In [18]:
tree = buildtree(my_data)

In [19]:
# show tree
def printtree(tree, indent=''):
    if tree.results != None:
        print(str(tree.results))
    else:
        # print condition
        print(str(tree.col) + ':' + str(tree.value) + ' ?')
        
        #print branch
        print(indent + 'T->', end='')
        printtree(tree.tb, indent + '  ')
        print(indent + 'F->', end='')
        printtree(tree.fb, indent + '  ')

In [20]:
printtree(tree)

0:google ?
T->3:21 ?
  T->{'Premium': 3}
  F->2:yes ?
    T->{'Basic': 1}
    F->{'None': 1}
F->0:slashdot ?
  T->{'None': 3}
  F->2:yes ?
    T->{'Basic': 4}
    F->3:21 ?
      T->{'Basic': 1}
      F->{'None': 3}


In [21]:
def getwidth(tree):
    if tree.tb == None and tree.fb == None: return 1
    return getwidth(tree.tb) + getwidth(tree.fb)

def getdepth(tree):
    if tree.tb == None and tree.fb == None: return 0
    return max(getdepth(tree.tb), getdepth(tree.fb)) + 1

In [36]:
from PIL import Image, ImageDraw

def drawtree(tree, jpeg='tree.jpeg'):
    w = getwidth(tree) * 100
    h = getdepth(tree) * 100 + 200
    
    img = Image.new('RGB', (w, h), (255, 255, 255))
    #Image.new(mode, size, color=0)
    draw = ImageDraw.Draw(img)
    
    drawnode(draw, tree, w/2, 20)
    img.save(jpeg, 'JPEG')

In [23]:
def drawnode(draw, tree, x, y):
    if tree.results == None:
        w1 = getwidth(tree.fb) * 100
        w2 = getwidth(tree.tb) * 100
        
        # total space
        left = x - (w1 + w2) / 2
        right = x + (w1 + w2) / 2
        
        # draw condition char
        draw.text((x-20, y-10), str(tree.col) + ':' + str(tree.value), (0,0,0))
        
        # draw branch line
        draw.line((x, y, left + w1/2, y+100), fill=(255, 0, 0))
        draw.line((x, y, right - w2/2, y+100), fill=(255, 0, 0))
        
        #draw node
        drawnode(draw, tree.fb, left + w1/2, y+100)
        drawnode(draw, tree.tb, right - w2/2, y+100)
    
    else: 
        txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
        draw.text((x-20, y), txt, (0,0,0))

In [37]:
drawtree(tree, jpeg='ch7_treeview.jpeg')

In [38]:
## classify new observations

def classify(observation, tree):
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v, int) or isinstance(v, float):
            if v >= tree.value: branch = tree.tb
            else: branch = tree.fb
        else:
            if v == tree.value: branch=tree.tb
            else: branch = tree.fb
    return classify(observation, branch)

In [39]:
classify(['(direct)', 'USA', 'yes', 5], tree)

{'Basic': 4}

In [40]:
classify(['google', 'USA', 'no', 25], tree)

{'Premium': 3}

In [41]:
# avoid overfitted use prning the tree

def prune(tree, mingain):
    # if not leaf node, prune
    if tree.tb.results == None:
        prune(tree.tb, mingain)
    if tree.fb.results == None:
        prune(tree.fb, mingain)
    
    # If both the subbranches are now leaves, see if they
    # combine
    if tree.tb.results != None and tree.fb.results != None:
        tb, fb = [], []
        for v, c in tree.tb.results.items():
            tb += [[v]] * c
        for v, c in tree.fb.results.items():
            fb += [[v]] * c
        
        delta = entropy(tb + fb) - (entropy(tb) + entropy(fb) / 2)
        if delta < mingain:
            tree.tb, tree.fb = None, None
            tree.results = uniquecounts(tb + fb)

In [42]:
tree = buildtree(my_data)

In [43]:
prune(tree, 0.1)
printtree(tree)

0:google ?
T->3:21 ?
  T->{'Premium': 3}
  F->2:yes ?
    T->{'Basic': 1}
    F->{'None': 1}
F->0:slashdot ?
  T->{'None': 3}
  F->2:yes ?
    T->{'Basic': 4}
    F->3:21 ?
      T->{'Basic': 1}
      F->{'None': 3}


In [44]:
prune(tree, 1)
printtree(tree)

0:google ?
T->3:21 ?
  T->{'Premium': 3}
  F->2:yes ?
    T->{'Basic': 1}
    F->{'None': 1}
F->{'Basic': 5, 'None': 6}


In [ ]:
# desicion tree advantage: ability dealing missing value

In [45]:
def mdclassify(observation, tree):
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        if v == None:
            tr, fr = mdclassify(observation, tree.tb), mdclassify(observation, tree.fb)
            tcount = sum(tr.values())
            fcount = sum(fr.values())
            tw = float(tcount) / (tcount + fcount)
            fw = float(fcount) / (tcount + fcount)
            result = {}
            for k, v in tr.items(): result[k] = v * tw
            for k, v in fr.items(): 
                if k not in result: result[k] = 0
                result[k] += v * fw
            return result
        else:
            if isinstance(v, int) or isinstance(v, float):
                if v >= tree.value: branch = tree.tb
                else: branch = tree.fb
            else:
                if v == tree.value: branch=tree.tb
                else: branch = tree.fb
            return mdclassify(observation, branch)

In [46]:
mdclassify(['google', None, 'yes', None], tree)

{'Basic': 0.25, 'Premium': 2.25}

In [47]:
mdclassify(['google', 'Franch', None, None], tree)

{'Basic': 0.125, 'None': 0.125, 'Premium': 2.25}

In [48]:
## looks not have big difference missing

In [49]:
mdclassify([None, 'Franch', 'yes', None], tree)

{'Basic': 4.12037037037037,
 'None': 4.888888888888888,
 'Premium': 0.41666666666666663}

In [50]:
#

def variance(rows):
    if len(rows) == 0: return 0
    data = [float(row[len(row) - 1]) for row in rows]
    mean = sum(data) / len(data)
    variance = sum([(d - mean) ** 2 for d in data]) / len(data)
    return variance

In [51]:
# dealing house price

def getpricelist():
    l1 = []
    for line in open('./DATA/addresslist.txt'):
        data = getaddressdata(line.strip(), 'Cambridge, MA')
        l1.append(data)
    return l1

In [52]:
housedata = getpricelist()
housetree = buildtree(housedata, scoref=variance)
printtree(housetree)

NameError: name 'getaddressdata' is not defined

In [ ]:
## Exercise:


In [ ]:
# E1: print result probility

def classify2(observation, tree):
    result = {}
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v, int) or isinstance(v, float):
            if v >= tree.value: branch = tree.tb
            else: branch = tree.fb
        else:
            if v == tree.value: branch=tree.tb
            else: branch = tree.fb
    return classify(observation, branch)